In [92]:
import numpy as np
from skimage.io import imread, imsave
import math

def adaptive_thresholding(im, part_len, part_wid):
        manip_im = im.copy()
        len_block = int(manip_im.shape[0]/part_len)
        width_block = int(manip_im.shape[1]/part_wid)
        
        for i in range(part_wid):
            for j in range(part_len):
                block = manip_im[i*len_block:(i+1)*len_block, j*width_block:(j+1)*width_block]
                varience = np.var(block)
                if varience <= 2000:
                    T = block.mean()
                    for k in range(block.shape[0]):
                        for l in range(block.shape[1]):
                            if block[k, l] > T:
                                manip_im[i*len_block+k, j*width_block+l] = 255
                            else:
                                manip_im[i*len_block+k, j*width_block+l] = 0
                else:
                    block = adaptive_thresholding(block, part_len, part_wid)
                    for k in range(block.shape[0]):
                        for l in range(block.shape[1]):
                            manip_im[i*len_block+k, j*width_block+l] = block[k, l]

        return manip_im
def rgb_to_grayscale_entropymine(im: np.ndarray):
        result = np.zeros((im.shape[0], im.shape[1]))

        for i in range(im.shape[0]):
            for j in range(im.shape[1]):
                r = im[i, j, 0]
                g = im[i, j, 1]
                b = im[i, j, 2]
                result[i, j] = (0.2126*r**2.2 + 0.7152*g**2.2 + 0.0722*b**2.2)**(1/2.2)
        return result

def nicblack_adaptive_thresholding(im, n, m, k=0.3): # n and m must be odd
    result = im.copy()
    rows = im.shape[0]
    cols = im.shape[1]
    for i in range(rows):
        for j in range(cols):
            box = []
            for l in range((2 * math.floor(n/2))+1):
                for o in range((2 * math.floor(m/2))+1):
                    curr_i = i - (math.floor(n/2) - l)
                    curr_j = j - (math.floor(m/2) - o)
                    if curr_i >= 0 and curr_i < rows and curr_j >= 0 and curr_j < cols:
                        box.append(im[curr_i, curr_j])
            np_box = np.array(box)
            mean = np_box.mean()
            std = np_box.std()
            T = mean + (k * std)
            if im[i, j] > T:
                result[i, j] = 255
            else:
                result[i, j] = 0
    return result

def sauvolas_adaptive_thresholding(im, n, m, k=0.34, R=128): # n and m must be odd
    result = im.copy()
    rows = im.shape[0]
    cols = im.shape[1]
    for i in range(rows):
        for j in range(cols):
            box = []
            for l in range((2 * math.floor(n/2))+1):
                for o in range((2 * math.floor(m/2))+1):
                    curr_i = i - (math.floor(n/2) - l)
                    curr_j = j - (math.floor(m/2) - o)
                    if curr_i >= 0 and curr_i < rows and curr_j >= 0 and curr_j < cols:
                        box.append(im[curr_i, curr_j])
            np_box = np.array(box)
            mean = np_box.mean()
            std = np_box.std()
            T = mean * (1+k * ((-1+std)/R))
            if im[i, j] > T:
                result[i, j] = 255
            else:
                result[i, j] = 0
    return result

def bernsens_adaptive_thresholding(im, n, m): # n and m must be odd
    im = im.astype(np.float64)
    result = im.copy()
    rows = im.shape[0]
    cols = im.shape[1]
    for i in range(rows):
        for j in range(cols):
            box = []
            for l in range((2 * math.floor(n/2))+1):
                for o in range((2 * math.floor(m/2))+1):
                    curr_i = i - (math.floor(n/2) - l)
                    curr_j = j - (math.floor(m/2) - o)
                    if curr_i >= 0 and curr_i < rows and curr_j >= 0 and curr_j < cols:
                        box.append(im[curr_i, curr_j])
            box.sort()
            T = 0.5 * (box[0] + box[1])
            if im[i, j] > T:
                result[i, j] = 255
            else:
                result[i, j] = 0
    return result

class Utils:
    def save(self, im: np.ndarray, filename = "save.jpg"):
        """Save image"""
        imsave(filename, im)

    def save_rgb(self, r, g, b, filename_r = "red.jpg", filename_g = "green.jpg", filename_b = "blue.jpg"):
        """Save r, g, b channels seperetly"""
        self.save(self.make(r, "red"), filename_r)
        self.save(self.make(g, "green"), filename_g)
        self.save(self.make(b, "blue"), filename_b)
    
    def make(self, l: np.ndarray, color: str):
        """Make a red, green, or blue only image"""
        new = np.zeros((l.shape[0], l.shape[1], 3))
        if color == "red":
            new[:, :, 0] = l
            return new
        elif color == "green":
            new[:, :, 1] = l
            return new
        elif color == "blue":
            new[:, :, 2] = l
            return new
        else:
            raise ValueError("expected a valid color as input")
    
    def split_rgb(self, im: np.ndarray):
        """Split an rgb"""
        return im[:, :, 0], im[:, :, 1], im[:, :, 2]
    
    def combine_rgb(self, r: np.ndarray, g: np.ndarray, b: np.ndarray):
        """Combine an rgb"""
        new = np.zeros((r.shape[0], r.shape[1], 3))
        new[:, :, 0] = r
        new[:, :, 1] = g
        new[:, :, 2] = b
        return new

    def generate_histogram(self, im: np.ndarray):
        """Generate a histogram"""
        return im.flatten()
    
    def plot_histogram(self, h: np.ndarray):
        """Simple histogram plot"""
        plt.hist(h, bins=256)
        plt.show()
    
    def histogram_distance(self, h1: np.ndarray, h2: np.ndarray):
        """Returns histogram distance"""
        sum = 0
        for i in range(h1.shape[0]):
            sum += abs(h1[i] - h2[i])
        return sum

In [10]:
imsave("interior3_gray.jpg", rgb_to_grayscale_entropymine(imread("interior3.jpg")))

Lossy conversion from float64 to uint8. Range [0.0, 255.00000000000006]. Convert image to uint8 prior to saving to suppress this warning.


In [28]:
imsave("interior3_gray_adaptive4.jpg", adaptive_thresholding(imread("interior3_gray.jpg"), 4, 4))

In [91]:
im = bernsens_adaptive_thresholding(imread("interior3_gray.jpg"), 7, 7)
imsave("interior3_gray_adaptive_bernsens7.jpg", im)

Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


In [93]:
utils = Utils()
im = imread("interior1.jpg")
split_r, split_g, split_b = utils.split_rgb(im)
ada_im_r = nicblack_adaptive_thresholding(split_r, 10, 10)
ada_im_g = nicblack_adaptive_thresholding(split_r, 10, 10)
ada_im_b = nicblack_adaptive_thresholding(split_r, 10, 10)
ada_im = utils.combine_rgb(ada_im_r, ada_im_g, ada_im_b)
imsave("interior3_gray_adaptive_nicblack_rgb.jpg", ada_im)

Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
